In [6]:
import os, sys 
import cv2
import numpy as np
import neuroglancer
from skimage import io, measure
import matplotlib.pyplot as plt

viewer = neuroglancer.Viewer()
print(viewer)

http://127.0.0.1:44693/v/baa453c140b3f013ecc5821b5b1c9fc1dadbdf1c/


In [7]:
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.imported_atlas_utilities import volume_to_polydata, save_mesh_stl

Connecting dklab@192.168.1.12:3306


In [2]:
def mask_to_shell(mask):
    sub_contours = measure.find_contours(mask, 1)

    sub_shells = []
    for sub_contour in sub_contours:
        sub_contour.T[[0, 1]] = sub_contour.T[[1, 0]]
        pts = sub_contour.astype(np.int32).reshape((-1, 1, 2))

        sub_shell = np.zeros(mask.shape, dtype='uint8')
        sub_shell = cv2.polylines(sub_shell, [pts], True, 1, 5, lineType=cv2.LINE_AA)
        sub_shells.append(sub_shell)
    shell = np.array(sub_shells).sum(axis=0)
    
    return shell

In [3]:
VOL_DIR = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/masks/MD594/aligned'
files = sorted(os.listdir(VOL_DIR))

volume = []
for file in files:
    tif = io.imread(os.path.join(VOL_DIR, file))
    tif = mask_to_shell(tif)
    volume.append(tif)
volume = np.array(volume).astype('uint8')
volume = np.swapaxes(volume, 0, 2)

In [23]:
volume.shape

(1468, 728, 433)

In [34]:
origin_shape = [-728/2, -1468/2, -433/2]
atlas_resolution = 1.0
origin = [o/atlas_resolution for o in origin_shape]
volume_origin = (volume, origin)
aligned_structure = volume_to_polydata(volume=volume_origin,
                           num_simplify_iter=3, smooth=True,
                           return_vertex_face_list=False)

In [35]:
filepath = os.path.join(HOME, 'tmp', 'meshes', 'MD94_shell.stl')
save_mesh_stl(aligned_structure, filepath)

In [ ]:
layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=[460, 460, 20000]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
#     s.layers.clear()
    s.layers['layer'] = layer